In [1]:
%matplotlib inline
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import tensorflow as tf
from sklearn import feature_extraction
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D,AveragePooling2D, GlobalMaxPool2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
def get_data(filename):
    img = nib.load(filename)
    return img.get_data(), img.affine
def scale(array):
    return (array-np.min(array))/(np.max(array) - np.min(array))
mri_data = get_data('case_001_2.nii.gz')[0].swapaxes(0,2)
labelled_data = get_data('tumor.nii.gz')[0].swapaxes(0,2)
shape = labelled_data.shape
mri_data = scale(mri_data.reshape((shape[0],shape[1],shape[2],1)))


In [3]:
# https://github.com/jocicmarko/ultrasound-nerve-segmentation/blob/master/train.py#L19
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [4]:
# output = 1 if slice has a tumor
y_train = np.sum(labelled_data,axis=(1,2))>0

In [9]:
model = Sequential()
model.add(Conv2D(20,(5,5),input_shape=(shape[1],shape[2],1), activation='relu'))
#model.add(MaxPooling2D(pool_size=(10,10)))
#model.add(Conv2D(20, (10, 10),activation='relu'))
model.add(MaxPooling2D(pool_size=(10,10)))
#model.add(GlobalMaxPool2D())
model.add(Dropout(0.25))
model.add(Flatten())
#model.add(Dense(20,activation='relu'))
model.add(Dense(30,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1,activation='softmax'))
model.compile(optimizer='rmsprop',
              loss=dice_coef_loss, metrics=[dice_coef,'accuracy'])
print(model.summary())
model.fit(mri_data,y_train,epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 316, 284, 20)      520       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 28, 20)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 31, 28, 20)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 17360)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 30)                520830    
_________________________________________________________________
dropout_10 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 31        
Total para

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 198, 198, 32)      320       
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 196, 196, 64)      18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 98, 98, 64)        0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 98, 98, 64)        0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 614656)            0         
_________________________________________________________________
dense_42 (Dense)             (None, 128)               78676096  
_________________________________________________________________
dropout_20 (Dropout)         (None, 128)               0         
__________

In [ ]:
predict = model.predict(mri_data)

In [16]:
y_train

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False], dtype=bool)